# Change Addresses to Longitude and Latitude
SUUMO.jp gives us addresses for each property. To convert it to spacial data, longitude and latitude are obtained each chome in each of the 23 inner wards of Tokyo from https://nlftp.mlit.go.jp/cgi-bin/isj/dls/_view_cities_wards.cgi. 

In [178]:
import pandas as pd
from pathlib import Path
import os

## Combine ward data
Combine the CSV longitude and latitude data from each ward.

In [179]:
if not os.path.exists("/app/data/geolocation.csv"):

    def combine_csv_files(directory_path, output_file):
        base_path = Path(directory_path)

        dataframes = []

        # Iterate over each CSV
        for csv_file in base_path.rglob('*.csv'):
            df = pd.read_csv(csv_file, encoding='shift_jis')
            dataframes.append(df)

        combined_df = pd.concat(dataframes, ignore_index=True)

        combined_df.to_csv(output_file, index=False, encoding='shift_jis')

        print(f"All CSV files have been combined into {output_file}")

    # Combine CSV files from 'wards' folder into 'geolocation.csv'
    combine_csv_files('/app/data/wards', '/app/data/geolocation.csv')


## Unique addresses
Get the unique addresses from the suumo property dataset.

In [180]:
# Read the CSV file into a pandas DataFrame
df = pd.read_csv('/app/data/sales_property_data.csv')
# df['address_block'] = df['address'].str.replace(r'[-－]\d+[-－]\d+', '', regex=True)
df['address_block'] = df['address'].str.extract(r'(.*?\d+)', expand=False)  # Extracting up to (and including) the first number
df['address_block'] = df['address_block'].fillna(df['address']) # .extract returns NaN if no match, fill NaN with original address
unique_addresses = df['address_block'].unique()
print(f"Total unique addresses: {len(unique_addresses)} of {len(df['address'])}")
df.head()

Total unique addresses: 2477 of 20972


,Category,URL,price,plan,area,balcony_area,level,no_floors,year,address,features,rooms,living_room,dining_room,kitchen,storage,address_block
0,used_apartments,https://suumo.jp/ms/chuko/tokyo/sc_adachi/nc_7...,11900000.0,2DK,41.34,2.43,2.0,4.0,1979,東京都足立区花畑５-2-9,年内引渡可 / 即引渡可 / 角住戸 / 陽当り良好 / 全居室収納 / シャワー付洗面化粧...,2,False,True,True,0,東京都足立区花畑５
1,used_apartments,https://suumo.jp/ms/chuko/tokyo/sc_taito/nc_75...,13800000.0,1K,16.45,2.69,5.0,9.0,1991,東京都台東区松が谷３-６－２,瑕疵保証付（不動産会社独自） / ２沿線以上利用可 / 内装リフォーム / 駅まで平坦 / ...,1,False,False,True,0,東京都台東区松が谷３
2,used_apartments,https://suumo.jp/ms/chuko/tokyo/sc_taito/nc_75...,29800000.0,2DK,43.13,4.58,9.0,12.0,1975,東京都台東区下谷３-16-14,修繕・点検の記録 / ２沿線以上利用可 / 年度内引渡可 / 省エネ給湯器 / スーパー 徒...,2,False,True,True,0,東京都台東区下谷３
3,used_apartments,https://suumo.jp/ms/chuko/tokyo/sc_koto/nc_741...,34900000.0,3LDK,64.96,7.53,4.0,14.0,1984,東京都江東区南砂１-19-1,スーパー 徒歩10分以内 / システムキッチン / セキュリティ充実 / ＴＶモニタ付インタ...,3,True,True,True,0,東京都江東区南砂１
4,used_apartments,https://suumo.jp/ms/chuko/tokyo/sc_sumida/nc_7...,41800000.0,2LDK,54.13,8.92,7.0,10.0,2011,東京都墨田区墨田２,２沿線以上利用可 / リバーサイド / スーパー 徒歩10分以内 / システムキッチン / ...,2,True,True,True,0,東京都墨田区墨田２


## Format suumo addresses
Remove double width numbers and replace them with the chome format in the longitude and latitude data.

In [181]:
def arabic_to_kanji(num):
    kanji_numerals = ["", "一", "二", "三", "四", "五", "六", "七", "八", "九"]
    digits = ["", "十", "百", "千"]
    kanji_string = ""
    num_str = str(num)
    position = 0

    for digit in reversed(num_str):
        if int(digit) > 0:
            kanji_string = kanji_numerals[int(digit)] + digits[position] + kanji_string
        position += 1

    # Special case handling for numbers like 10, 20, 30, etc.
    kanji_string = kanji_string.replace("一十", "十")
    
    return kanji_string

def convert_address_format(address):
    import re
    # Find all numbers in the address
    numbers = re.findall(r'\d+', address)
    
    for number in numbers:
        kanji_number = arabic_to_kanji(int(number)) + '丁目'
        # Replace the original number with its kanji representation
        address = address.replace(number, kanji_number, 1)
    
    return address


df['converted_address'] = df['address_block'].apply(convert_address_format)
df['converted_address'] = df['converted_address'].str.strip()
df['converted_address'] = df['converted_address'].str.replace('ヶ', 'ケ')
print(df[['address_block', 'converted_address']])


      address_block converted_address
0         東京都足立区花畑５       東京都足立区花畑五丁目
1        東京都台東区松が谷３      東京都台東区松が谷三丁目
2         東京都台東区下谷３       東京都台東区下谷三丁目
3         東京都江東区南砂１       東京都江東区南砂一丁目
4         東京都墨田区墨田２       東京都墨田区墨田二丁目
...             ...               ...
20967    東京都新宿区西新宿６      東京都新宿区西新宿六丁目
20968    東京都渋谷区代々木５      東京都渋谷区代々木五丁目
20969    東京都世田谷区玉川１      東京都世田谷区玉川一丁目
20970      東京都港区白金６        東京都港区白金六丁目
20971      東京都港区赤坂９        東京都港区赤坂九丁目

[20972 rows x 2 columns]


In [182]:
file_path = '/app/data/geolocation.csv'

jp_geo_df = pd.read_csv(file_path, encoding='shift_jis')
jp_geo_df.head()

,都道府県コード,都道府県名,市区町村コード,市区町村名,大字町丁目コード,大字町丁目名,緯度,経度,原典資料コード,大字・字・丁目区分コード
0,13,東京都,13121,足立区,131210001000,千住橋戸町,35.740193,139.797811,0,1
1,13,東京都,13121,足立区,131210002001,千住緑町一丁目,35.742577,139.792502,0,3
2,13,東京都,13121,足立区,131210002002,千住緑町二丁目,35.744936,139.792197,0,3
3,13,東京都,13121,足立区,131210002003,千住緑町三丁目,35.747166,139.792460,0,3
4,13,東京都,13121,足立区,131210003000,千住河原町,35.743103,139.798787,0,1


In [183]:
# jp_geo_df['combined'] = (jp_geo_df['都道府県名'].str.strip() + 
#                          jp_geo_df['市区町村名'].str.strip() + 
#                          jp_geo_df['大字町丁目名'].str.strip())
# # column_values = jp_geo_df['combined'].tolist()
# # column_values = [value for value in jp_geo_df['combined'] if '十' in value]
# column_values = [value for value in jp_geo_df['combined'] if '東京都新宿区内' in value]

# for c in column_values:
#     print(c)
# # print(column_values)

## Format addresses in geolocation and join
The addresses in the geolocation data are split into 3 columns. First combine these columns so the addresses in the two datasets match and then merge the logitude and latitude columns into the original dataset using the matching addresses.

In [184]:
jp_geo_df['combined'] = (jp_geo_df['都道府県名'].str.strip() + 
                         jp_geo_df['市区町村名'].str.strip() + 
                         jp_geo_df['大字町丁目名'].str.strip())

# Performing a left join on the address column
result_df = pd.merge(df, jp_geo_df[['combined', '緯度', '経度']], left_on='converted_address', right_on='combined', how='left')
result_df.rename(columns={'緯度': 'latitude', '経度': 'longitude'}, inplace=True)
result_df.drop(["combined"], axis=1, inplace=True)

# print(result_df[['converted_address', 'latitude', 'longitude']])

## NaN longitude and latitude
The longitude and latitude are NaN in some cases. This is because the geolocation data does not cover some blocks. To deal with this, exclude the final number specifying the block. This results in a less accurate geolocation as the logitude and latitude are of a small area rather than the exact block.

In [185]:
# Find rows where latitude is NaN
mask = result_df['latitude'].isnull()

# Temporary extraction of modified addresses
result_df['temp_address'] = result_df['address'].str.extract(r'(.*?)(?=\d+)', expand=False)

# Match the modified address and update latitude and longitude
for idx, row in result_df.loc[mask].iterrows():
    match = jp_geo_df[jp_geo_df['combined'] == row['temp_address']]
    if not match.empty:
        result_df.at[idx, 'latitude'] = match['緯度'].values[0]
        result_df.at[idx, 'longitude'] = match['経度'].values[0]

# Drop the temporary address column after use
result_df.drop(columns=['temp_address'], inplace=True)

## Save the combined dataset
Save the combined dataset and print out the number of missing values in latitude and longitude

In [186]:
result_df.to_csv('/app/data/sales_combo_property_geo.csv', index=False)

# Check if any latitude or longitude values are missing
missing_lat = result_df['latitude'].isna().any()
missing_lon = result_df['longitude'].isna().any()

if missing_lat or missing_lon:
    print("There are missing values in latitude or longitude.")
else:
    print("All rows have latitude and longitude.")

# Count missing values
missing_lat_count = result_df['latitude'].isna().sum()
missing_lon_count = result_df['longitude'].isna().sum()

print(f"Missing latitude values: {missing_lat_count}")
print(f"Missing longitude values: {missing_lon_count}")

All rows have latitude and longitude.
Missing latitude values: 0
Missing longitude values: 0
